In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
!nvidia-smi



Tue May 18 19:29:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = '/content/drive/My Drive/Dataset/dataset_new.zip'
!unzip "/content/drive/My Drive/Dataset/dataset-18-5-21.zip" -d "/content"

Archive:  /content/drive/My Drive/Dataset/dataset-18-5-21.zip
   creating: /content/images/
   creating: /content/images/without_mask/
  inflating: /content/images/without_mask/0_0_aidai_0014.jpg  
  inflating: /content/images/without_mask/0_0_aidai_0029.jpg  
  inflating: /content/images/without_mask/0_0_aidai_0043.jpg  
  inflating: /content/images/without_mask/0_0_aidai_0074.jpg  
  inflating: /content/images/without_mask/0_0_aidai_0084.jpg  
  inflating: /content/images/without_mask/0_0_aidai_0136.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0004.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0020.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0025.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0027.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0056.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0057.jpg  
  inflating: /content/images/without_mask/0_0_anhu_0062.jpg  
  inflating: /content/images/without_mask/0_0_anhu_00

In [ ]:
images = '/content/images'
ImagePaths = list(paths.list_images(images))



In [ ]:
ImagePaths

['/content/images/without_mask/cropped1019.jpg',
 '/content/images/without_mask/1_0_chenxiang_0032.jpg',
 '/content/images/without_mask/0_0_liyifeng_0051.jpg',
 '/content/images/without_mask/cropped617.jpg',
 '/content/images/without_mask/0_0_jianailiang_0006.jpg',
 '/content/images/without_mask/1_0_lixingliang_0156.jpg',
 '/content/images/without_mask/1_0_luojin_0105.jpg',
 '/content/images/without_mask/1_0_fanwei_0009.jpg',
 '/content/images/without_mask/1_0_anhu_0088.jpg',
 '/content/images/without_mask/cropped840.jpg',
 '/content/images/without_mask/1_0_anhu_0122.jpg',
 '/content/images/without_mask/cropped815.jpg',
 '/content/images/without_mask/cropped85.jpg',
 '/content/images/without_mask/0_0_chenxuedong_0070.jpg',
 '/content/images/without_mask/1_0_liuqianhan_0050.jpg',
 '/content/images/without_mask/1_0_luojin_0007.jpg',
 '/content/images/without_mask/cropped122.jpg',
 '/content/images/without_mask/1_0_anhu_0094.jpg',
 '/content/images/without_mask/cropped1088.jpg',
 '/conten

In [ ]:
data=[]
labels=[]

for i in ImagePaths:
    label = i.split('/')[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image = img_to_array(image)
    image=preprocess_input(image)
    data.append(image)


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
data = np.array(data,dtype='float32')
labels = np.array(labels)
labels


array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [ ]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)


In [ ]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(data, labels, test_size=0.20, random_state=42,stratify=labels)


In [ ]:
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.20,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [ ]:
baseModel = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))


9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
baseModel.trainable = False

In [ ]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
learning_rate = 1e-4
Epochs = 20
Batch_size = 32

opt = Adam(lr=learning_rate,decay=learning_rate//Epochs)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
History = model.fit(
	aug.flow(train_X, train_Y, batch_size=Batch_size),
	steps_per_epoch=len(train_X) // Batch_size,
	validation_data=(test_X, test_Y),
	validation_steps=len(test_X) // Batch_size,
	epochs=Epochs)

Epoch 1/20
99/99 [==============================] - 65s 311ms/step - loss: 0.6065 - accuracy: 0.7364 - val_loss: 0.1774 - val_accuracy: 0.9525
Epoch 2/20
99/99 [==============================] - 30s 298ms/step - loss: 0.1739 - accuracy: 0.9586 - val_loss: 0.1156 - val_accuracy: 0.9650
Epoch 3/20
99/99 [==============================] - 30s 299ms/step - loss: 0.1257 - accuracy: 0.9634 - val_loss: 0.0963 - val_accuracy: 0.9688
Epoch 4/20
99/99 [==============================] - 30s 300ms/step - loss: 0.1030 - accuracy: 0.9700 - val_loss: 0.0869 - val_accuracy: 0.9737
Epoch 5/20
99/99 [==============================] - 30s 300ms/step - loss: 0.0930 - accuracy: 0.9660 - val_loss: 0.0839 - val_accuracy: 0.9700
Epoch 6/20
99/99 [==============================] - 30s 299ms/step - loss: 0.0856 - accuracy: 0.9722 - val_loss: 0.0806 - val_accuracy: 0.9725
Epoch 7/20
99/99 [==============================] - 30s 299ms/step - loss: 0.0856 - accuracy: 0.9741 - val_loss: 0.0775 - val_accuracy: 0.9750

In [ ]:
pred = model.predict(test_X, batch_size=Batch_size)
pred = np.argmax(pred, axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_Y.argmax(axis=1), pred,
	target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.98      0.98      0.98       385
without_mask       0.98      0.98      0.98       415

    accuracy                           0.98       800
   macro avg       0.98      0.98      0.98       800
weighted avg       0.98      0.98      0.98       800



In [ ]:
model.save("Mask Detection_5-18-2021",save_format='h5')

In [ ]:
import cv2
img = cv2.imread("/content/test.jpg")
resized_img = cv2.resize(img, (224,224))
resized_img = img_to_array(resized_img)
resized_img = np.reshape(resized_img,(-1,224,224,3))
resized_img = preprocess_input(resized_img)
pred = model.predict(resized_img)

error: ignored

In [ ]:
print(pred)
label=np.argmax(pred,axis=1)[0]
print(np.amax(pred,axis=1)[0]*100)
label

[[9.9999976e-01 2.8203164e-07]]
99.99997615814209


0